## Basic calcium imaging analysis using Jupyter, Spark and Thunder

### General Introduction

The Jupyter Notebook is an interactive web application for creating and sharing documents that contain live code, equations, visualizations and explanatory text. As a web application, it can be run simply within the browser while the underlying computations are either performed on the local computer, a remote server or even a remote cluster. Therefore, this approach to data analysis scales very well from small to large datasets: we can start by analysing a small part of our data set on the local machine and, once we are satisfied with the results, easily scale up to analyse the full data set on a cluster. The Jupyter notebook can also be used to share data and analysis with others. This can be useful e.g. for teaching or for providing data / analysis in the context of a publication. See for example [http://www.nature.com/news/ipython-interactive-demo-7.21492](http://www.nature.com/news/ipython-interactive-demo-7.21492) or [https://github.com/ipython/ipython/wiki/A-gallery-of-interesting-IPython-Notebooks#reproducible-academic-publications](https://github.com/ipython/ipython/wiki/A-gallery-of-interesting-IPython-Notebooks#reproducible-academic-publications).



### Initial setup

In [ ]:
# Import required modules
import numpy as np
import matplotlib.pyplot as plt
import h5py
import os, sys
import seaborn as sns

# Set figure style options for Seaborn
sns.set_style('darkgrid')
sns.set_context('notebook')

# show figure in notebook
%matplotlib inline

In [ ]:
# add folder 'utils' to the Python path
# this folder contains custom written code that is required for data import and analysis
utils_dir = os.path.join(os.getcwd(), 'utils')
sys.path.append(utils_dir)

In [ ]:
# starting Spark depends on where the notebook is running (local computer or OpenStack cluster)
# choose 'local' or 'openstack'
nbBackend = 'openstack'
print "Running notebook on " + nbBackend + " backend"

In [ ]:
# Initialize Spark
# returns the SparkContext object 'sc' which tells Spark how to access the cluster
from setupSpark import initSpark
sc = initSpark(nbBackend)

In [ ]:
# add Python files in 'utils' folder to the SparkContext 
# this is required so that all files are available on all the cluster workers
for filename in os.listdir(utils_dir):
    if filename.endswith('.py'):
        sc.addPyFile(os.path.join(utils_dir, filename))

### Import data

The tutorial dataset consists of four example two-photon calcium imaging recordings performed by Frauke Leitner in the lab of Hannah Monyer at the DKFZ in Heidelberg. Recordings were obtained in the mouse lateral entorhinal cortex (LEC) while odor stimulation (6 different odors + air) was applied to the mouse nose. Calcium imaging was performed at ca. 5 Hz using standard frame scans. The calcium indicator is GCamp6, delivered by viral injection. Further details about the data set are described in the corresponding [paper](http://www.nature.com/neuro/journal/v19/n7/full/nn.4303.html).

For the tutorial, raw data have already been background subtracted and Roi time series have been extracted and converted to DFF. The data are stored in HDF5 files, with one file per imaging area. For each trial, the file contains image data (2D matrix with different Rois' time series in separate rows), Roi coordinates, Roi names, a reference image of the population as well as stimulation information. Note that HDF5 files can be conveniently viewed using the free utility [HDFView](https://www.hdfgroup.org/products/java/hdfview/). 

In [ ]:
# full path to directory containing HDF5 files
directory = '/home/ubuntu/example_data/LEC_Data'

# select HDF5 file
# following files are available: 
# Monyer_Leitner_F296_spot01.h5
# Monyer_Leitner_F397_spot01.h5
# Monyer_Leitner_F400_spot02.h5
# Monyer_Leitner_F400_spot04.h5
h5file = 'Monyer_Leitner_F296_spot01.h5'
h5file = directory + os.sep + h5file

In [ ]:
# First, we obtain some information about the dataset (size, sampling rate, number of trials)
# NeuroH5Utils is a python file in the 'utils' folder which contains functions for reading the HDF5 files
from NeuroH5Utils import getFileInfo
dsetSz, sampF, nTrials = getFileInfo(h5file)

In [ ]:
# Next, we read the entire timeseries for one neuron to demonstrate the basic principle how data is read
ix = 0 # Neuron index (zero-based in Python)
from NeuroH5Utils import readPixel_map
x, result = readPixel_map(ix, h5file, dim=1, debug=True) # debug=True will plot the data

In [ ]:
# We can also read the activity of all neurons at one timepoint by setting dim=2
ix = 100
from NeuroH5Utils import readPixel_map
x, result = readPixel_map(ix, h5file, dim=2)

In [ ]:
# Spark revolves around the concept of a resilient distributed dataset (RDD), which is a 
# collection of elements that can be operated on in parallel.
# Thus, data has to be converted (parallelized) into an RDD
# This is done with the function convert2RDD which calls readPixel_map on every neuron in the dataset
from NeuroH5Utils import convert2RDD
numPartitions = 10 # how many partitions?
rdd = convert2RDD(sc, h5file, numPartitions=numPartitions)

In [ ]:
# Spark uses 'lazy' execution, i.e. data is only accessed when its actually needed
# We can use the count method to force loading of the data (this accesses every element once)
# count returns number of elements in the RDD (i.e. dsetSz[0])
nNeurons = rdd.count()
nNeurons

In [ ]:
# Get time series from first roi to derive time axis and number of timepoints
s  = np.asarray(rdd.lookup(0))
t = (np.linspace(1, len(s[0]), len(s[0]))) / sampF 
nTimepoints = len(t)

In [ ]:
# Finally, let's return a specific Roi as Python list / numpy array and plot the timeseries
roi = 0
s = rdd.lookup(roi) # returns a list
s = np.asarray(s) # convert to np array (actually not required for plotting)
plt.plot(t, s[0]);
plt.xlim((0, np.max(t)));
plt.xlabel('Time / s');

### From Spark to Thunder

Thunder is a collection of tools for the analysis of image and time series data in Python. It runs locally or against a Spark cluster. For more information on Thunder see [http://thunder-project.org/](http://thunder-project.org/) and the associated [paper](http://www.nature.com/nmeth/journal/v11/n9/full/nmeth.3041.html). 
The basic data types of Thunder are Images and Series objects. In the following, we will use the Series object to parallelize the timeseries from different neurons.

In [ ]:
# convert the Spark RDD into a Thunder Series object
# this allows us to make use of the functions available for Series objects in the Thunder library
import thunder as td
# thunder fromrdd expects a key-value pair where the key is a tuple representing the index
# here the keys are integers, so we wrap them in a single-element tuple
series = td.series.fromrdd(rdd.map(lambda kv: ((kv[0],), kv[1])))

In [ ]:
# Thunder provides a number of convenient functions for analyzing and processing the data
# In this example, we filter the traces to select those with a mean intensity > 10% DFF. 
# Then, we select a subset of 5 traces and convert them to numpy arrays for plotting.
# The example also demonstrates the use of lambda functions, a key technique in Python.
examples = series.filter(lambda x: x.mean() > 10).sample(5).toarray()
plt.plot(t, examples.T);
plt.xlim((0, np.max(t)));
plt.xlabel('Time / s');
plt.ylabel('% DFF');

In [ ]:
# In a similar way, compute the mean and standard deviation for each Roi
series_mean = series.map(lambda x: x.mean()).flatten().toarray()
series_sd = series.map(lambda x: x.std()).flatten().toarray()

In [ ]:
# Scatter plot of Roi mean vs. SD
plt.scatter(series_mean, series_sd)
plt.xlabel('Roi Mean');
plt.ylabel('Roi SD');

### Stimulation data

In most cases, neural data is acquired in the presence of sensory stimulation or together with recording of behavioral data. For our example data set, odor stimulation (6 different odors + air) was applied to the mouse nose. Next, we import the stimulus data from the HDF5 file.

In [ ]:
# import stimulus data
# getStimData is a function defined in the NeuroH5Utils.py file
from NeuroH5Utils import getStimData
stimData, stimNames = getStimData(h5file)

In [ ]:
# Now we can plot the timeseries for all neurons in stacked fashion and indicate the start of odor application
# In principle, this could be done with both Thunder series objects and Spark RDDs
# Based on non-comprehensive tests, it seems faster to extract data directly from Spark RDD
fig = plt.figure(figsize=(8,4)) # increase figsize to (20,10) to improve visibility
# loop over neurons to plot their respective timeseries
offset = 0
for iNeuron in range(nNeurons):
    plotTrace = rdd.lookup(iNeuron) # returns a list
    plotTrace = np.asarray(plotTrace) # convert to np array (actually not required for plotting)
    plotTrace = plotTrace[0] - min(plotTrace[0]) + offset
    offset = max(plotTrace)
    plt.plot(t, plotTrace)
# loop over stims and indicate them with vertical dashed lines
for iTimepoint in range(len(stimData)):
    if stimData[iTimepoint]:
        tStim = t[iTimepoint]
        plt.plot((tStim, tStim), (0, offset), 'k--')
# some improvements to the default figure
plt.xlabel('Time [s]', fontsize=18)
plt.ylim((0, offset))
plt.xlim((0, np.max(t)))
ax = fig.gca()
plt.setp(ax.get_xticklabels(), fontsize=16)
plt.setp(ax.get_yticklabels(), fontsize=16)
plt.show()
# save the figure if required
# plt.savefig('Timeseries_AllStim.png')

In [ ]:
# The previous plot shows the 'raw' activity level in relation to odor stimulation
# However, we might also be interested to know how the activity of a neuron changes on average for different odors
# This can be done with the peri-stimulus plot - the average trace for each stimulus per neuron
from CalciumAnalysisUtils import psAnalysis

# select time interval to plot (in frames)
baseFrames = 10
evokedFrames = 100

# compute peri-stimulus data for all neurons from the Spark RDD
# this creates a new RDD called psData
# rdd.map applies a function (in this case psAnalysis to all elements of the RDD in parallel)
psData = rdd.map(lambda (k, v): (k, psAnalysis(v, stimData, (baseFrames, evokedFrames))))
psData = psData.partitionBy(numPartitions).cache()

In [ ]:
# now create the plot

# select Rois to plot
roisToPlot = [2, 8, 12, 16, 19]
# Or select all neurons
# roisToPlot = range(nNeurons)

fig = plt.figure(figsize=(20,20)) # (20, 200) for full dataset, otherwise fewer rows
splotCounter = 1
for ix, iRoi in enumerate(roisToPlot):
    iRoi_data = np.asarray(psData.lookup(iRoi))
    psDataByStim = iRoi_data[0]
    # same y range for all stims
    minY = min([ np.min(x) for x in psDataByStim ])
    maxY = max([ np.max(x) for x in psDataByStim ])
    # plot for each stimulus
    for ix2, iStim in enumerate(psDataByStim):
        meanData = np.mean(iStim,axis=0)
        semData = np.std(iStim,axis=0) / np.sqrt(np.shape(iStim)[0])
        tPs = (np.linspace(0, evokedFrames, meanData.size)-baseFrames)/sampF
        plt.subplot(len(roisToPlot), len(psDataByStim), splotCounter)
        splotCounter = splotCounter + 1
        plt.fill_between(tPs, meanData-semData, meanData+semData, alpha=0.2)
        plt.plot(tPs, meanData)
        plt.plot((0,0), (minY, maxY), 'k--')
        plt.xlim((min(tPs), max(tPs)))
        plt.ylim((minY, maxY))
        if ix == 0:
            plt.title(stimNames[ix2+1])
        if ix2 == 0:
            plt.ylabel('%DFF Roi {0}'.format(iRoi+1))
plt.show()
# plt.savefig('PsPlot_AllStims.eps')

### Display image of population

In [ ]:
# Finally, let's get the reference image for this population and display it
from NeuroH5Utils import getReferenceImage
from showit import image
trial = 0 # specify trial (0 based indexing)
refImage = getReferenceImage(h5file, trial=trial)
image(refImage, clim=(0,80))